# Создание оптимального портфеля

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly
import cufflinks
import gurobipy as gp
from gurobipy import GRB
from calculator import Calculator

### Загрузка данных и создание матрицы доходностей

In [2]:
data = pd.read_csv('results.csv')  # полные данные (включая прогноз)
data = data.drop('Unnamed: 0', axis = 1)
data = data.set_index('ds')
data = data[986:]  # оставляем только прогноз, чтобы строить оптимальный портфель на будущее 

In [3]:
DJI_tickers = ['MMM',
               'AXP', 'AAPL', 'BA', 'CAT', 'CVX',
               'CSCO', 'KO', 'DOW', 'XOM', 'GS',
               'HD', 'IBM', 'INTC', 'JNJ', 'JPM',
               'MCD', 'MRK', 'MSFT', 'NKE', 'PFE',
               'PG', 'TRV', 'UNH', 'RTX', 'VZ',
               'V', 'WMT', 'WBA', 'DIS']

df = pd.DataFrame()

# получаем матрицу доходностей
for i in DJI_tickers: 
    dr = Calculator(data[i]).expected_asset_profit()
    df[f'{i}'] = dr

In [4]:
df.head()

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DOW,XOM,...,PFE,PG,TRV,UNH,RTX,VZ,V,WMT,WBA,DIS
0,-0.001154,-0.006546,0.006605,-0.015280,0.000647,-0.009423,-0.003508,-0.001300,-0.002374,-0.012941,...,-0.000517,0.000983,-0.003263,-0.000382,-0.028352,-0.001936,-0.000356,-0.002447,-0.009777,-0.003169
1,-0.001551,-0.003499,0.001016,-0.009718,0.001459,-0.004232,-0.002920,-0.001739,0.000046,-0.005546,...,-0.001677,0.000866,-0.001665,-0.000168,-0.013927,0.000547,-0.000665,-0.000696,-0.009635,-0.002394
2,0.000616,-0.002916,-0.002200,-0.008868,0.002579,-0.002755,-0.000499,-0.000017,0.000731,-0.007652,...,0.002948,0.002390,0.000588,0.000003,-0.014991,0.001874,-0.000430,-0.000843,-0.005479,-0.000757
3,-0.001729,-0.011439,-0.006461,-0.028647,0.005399,-0.005764,-0.004024,-0.001738,0.006410,-0.015016,...,-0.002610,0.000984,-0.003496,-0.005223,-0.052362,-0.002603,-0.003132,-0.002785,-0.015260,-0.003753
4,-0.000637,-0.001749,0.001064,-0.005150,0.003363,0.000570,0.001581,0.001250,0.006444,-0.004941,...,-0.001687,0.001561,-0.000981,0.001218,-0.008256,0.000551,0.000816,0.001808,-0.004209,0.000289


### Построение оптимизационной модели с помощью инструментов Gurobi

In [6]:
mod = gp.Model("Portfolio")  # создаем оптимизационную модель

In [7]:
vars = pd.Series(mod.addVars(data.columns, name=DJI_tickers), index=data.columns)  # добавляем 30 переменных

In [8]:
portfolioRisk = Calculator(df).portfolio_risk(vars)  # рассчитываем риск портфеля

In [9]:
mod.addConstr(vars.sum() == 1, 'budget')  # добавляем ограничение

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# ставим задачу минимизации риска и решаем её

mod.setParam('OutputFlag', 0)
mod.setObjective(portfolioRisk, GRB.MINIMIZE)
mod.optimize()

### Вывод результатов

In [11]:
print('Minimum risk portfolio: ')
for i in vars:
    print(i.varname, i.x)
print('\n')
print('Risk: ', np.sqrt(portfolioRisk.getValue()))
print('Return: ', df.mean().dot(vars).getValue() * 22)

Minimum risk portfolio: 
MMM 5.6182716547055304e-05
AXP 2.910139877631223e-05
AAPL 8.161583691075554e-06
BA 5.570806757087231e-06
CAT 0.07162997933449342
CVX 2.0714436416929903e-05
CSCO 1.2791299337435949e-05
KO 4.388277573734965e-05
DOW 1.201515038387751e-05
XOM 1.0528908952086468e-05
GS 2.3876489822332475e-05
HD 4.040534294288636e-05
IBM 2.5983305393889415e-05
INTC 2.3080076390897815e-05
JNJ 2.0756927069752782e-05
JPM 0.035222852943743325
MCD 3.866547648211571e-05
MRK 0.016132834686702277
MSFT 0.8097495324589286
NKE 0.0002788675049979015
PFE 0.023864932961270215
PG 5.769381243315736e-05
TRV 0.0003434932505729316
UNH 2.589068879384508e-05
RTX 5.864590211312071e-06
VZ 3.903217808616809e-05
V 2.3185168154569915e-05
WMT 5.5842529829677376e-05
WBA 8.633210164049328e-06
DIS 0.042189647986924754


Risk:  0.0012542544057195892
Return:  0.016742313923185585


### Вывод

Как можно заметить, риск построенного портфеля составляет 0.1%, а доходность около 1.7%. Если мы распределим свои 100 долларов по акциям компаний, входящих в индекс Доу-Джонса следующим образом, то прогнозируется, что к концу месяца мы получим 101.7 долларов с вероятностью почти 100%.

- Micrisoft - 81%
- Caterpillar Inc. - 7.1%
- Walt Disney Co. - 4.2%	
- JPMorgan Chase & Co. - 3.5%
- Pfizer Inc. - 2.4%
- Merck & Co. Inc. - 1.6%